In [27]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import seaborn
import pandas as pd
#import cv2
import scipy.optimize as op
import sys
import networkx as nx
import time
import matplotlib
import copy as cp
import scipy.ndimage as ndi 
from scipy.ndimage import gaussian_filter1d
import scipy as sp
import scipy.signal
seaborn.set(style='ticks',font_scale=1.5)

sys.path.append('/Users/yves/Documents/Code/mec_ephys/packages')


In [28]:
sys.path.append('/Users/yves/Documents/Code/mec_ephys/packages')

from mecll.rsync import Rsync_aligner
from mecll.proc_beh import get_all_transition, get_transitions_state, build_poke_df
from mecll.proc_neural import get_all_resps, get_mean_resps
from mecll.load import load_data
from mecll.SVD_analysis import get_mean_activity_matrix
from mecll.utils import align_activity

In [29]:
root_path = '/Users/yves/Downloads/example_files/'
out = load_data(root_path)
spkT,spkC,single_units,events,lines,aligner = out


single_unit_spkC = np.isin(spkC,single_units)


In [30]:
df = build_poke_df(lines,events)

In [31]:
df['previous_port'][1:] = df['port'][:-1]
df['previous_state'][1:] = df['state'][:-1]

In [32]:
spkT_ms = aligner.A_to_B(spkT)
spk_join = np.vstack([spkC,spkT_ms])[:,single_unit_spkC]

In [24]:
poke_dict_t1 ={}
poke_dict_t2 = {}

dicts = [{} for _ in range(4)]

for port_nr in np.unique(df['state'].values):
    
    kk = 0
    for task_nr in range(2):
        for direction in np.unique(df['direction'].values):
            task_nr = str(task_nr)
            indices = np.where((df['state']==port_nr) &
                               (df['correct']==True) & 
                               (df['direction']==direction) &
                               (df['port_repeat']==False) & 
                               (df['task_nr']==task_nr))[0]
            indices = indices[indices>0]
            v = df.loc[indices]['time'].values
            prev_v = df.loc[indices-1]['time'].values
            #print(direction,df.loc[indices]['state'].values - df.loc[indices-1]['state'].values)

            
            dicts[kk][str(port_nr)] = [[float(j),float(i)] for i,j in zip(v,prev_v) if (i-j)<800]
            dicts[kk]['task_nr'] = str(task_nr)
            dicts[kk]['direction'] = str(direction)
            dicts[kk]['graph_type'] = df.loc[df['task_nr']==task_nr]['graph_type'].values[0]
            dicts[kk]['seq'] = df.loc[df['task_nr']==task_nr]['current_sequence'].values[0]
            kk += 1


In [14]:
n_timepoints = 25
n_neurons = len((list(set(spk_join[0]))))

all_big_array = []
all_big_store = []
for dict_ix in[0,1]:
    
    big_array = np.zeros([n_neurons,n_timepoints*9])
    big_store = []
    for state_nr in range(9):

        direction = dicts[dict_ix]['direction']
        trial_events = np.array(dicts[dict_ix][str(state_nr)])
        print(len(trial_events))
        if len(trial_events)>0:
            out = align_activity(trial_events,np.array([0,1000]),spk_join,fs=25)
            #if len
            #algined_rates.shape = [n_trials, n_neurons, n_timepoints]
            aligned_rates, t_out, min_max_stretch = out

            mean_aligned_rates = np.nanmean(aligned_rates,axis=0)
        else:
            mean_aligned_rates = np.zeros([n_neurons,n_timepoints]) + np.nan
        if direction=='-1':
            big_array[:,state_nr*n_timepoints:(state_nr+1)*n_timepoints] = np.fliplr(mean_aligned_rates)
            big_store.append(np.flip(aligned_rates,axis=2))
        if direction=='1':
            big_array[:,(1+state_nr-1)*n_timepoints:(1+state_nr)*n_timepoints] = mean_aligned_rates
            big_store.append(aligned_rates)

    all_big_array.append(big_array)
    all_big_store.append(big_store)

9
12
15
11
14
11
10
13
0
0
12
14
16
15
19
16
13
9


In [21]:
#all_big_store.shape = (n_directions x n_states x n_trials x n_neurons x n_timepoints)
#not an array as has different numbers of trials for each
len(all_big_store[0][2][0])

320

In [23]:
classes = []
for state_arr in all_big_store[0]:
    state_arr

(15, 320, 25)

In [25]:
from sklearn.naive_bayes import GaussianNB